In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

import os
import time
import datetime
import random
import gc

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

from transformers import AutoModelForSequenceClassification

from transformers import AutoConfig
from transformers import AdamW
from transformers import AutoTokenizer
from transformers import get_linear_schedule_with_warmup

import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
import IPython

import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:21"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [25]:
df = pd.read_csv('data/df_folds.csv')
#neg_class = pd.read_csv('negative_class.csv')
#df = pd.concat([df,neg_class]).reset_index(drop=True)
#df['is_offensive'] = df['is_offensive'].fillna(0)
#df['fold'] = df['fold'].fillna(6)

In [26]:
#df = pd.read_csv('data/df_folds.csv')
df['text_len'] = df.text.progress_apply(lambda x: len(x))
df = df[df.text_len>=3]
df = df[~((df.target=='OTHER')&(df.is_offensive==1))]
df = df.reset_index(drop=True)
target_dict = {'OTHER':0,'PROFANITY':1,'SEXIST':2,'RACIST':3,'INSULT':4}
df['target_new'] = df['target'].map(target_dict)


X_train = df[df.fold != 0]['text']
X_val = df[df.fold == 0]['text']

y_train = df[df.fold != 0]['target_new']
y_val = df[df.fold == 0]['target_new']

  0%|          | 0/12617 [00:00<?, ?it/s]

In [27]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification,BertForSequenceClassification, BertTokenizer,XLMRobertaForSequenceClassification,XLMRobertaTokenizer
from transformers import get_cosine_schedule_with_warmup, DataCollatorWithPadding
from sklearn.metrics import f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [28]:
models = ['dbmdz/convbert-base-turkish-mc4-uncased',
          'dbmdz/electra-base-turkish-mc4-uncased-generator',
          'microsoft/deberta-base',
          'akdeniz27/bert-turkish-text-classification',
          'dbmdz/bert-base-turkish-128k-uncased',
          'dbmdz/bert-base-turkish-128k-cased',
         'dbmdz/bert-base-turkish-cased',
         'dbmdz/convbert-base-turkish-mc4-uncased']

In [30]:
class CFG:
    print_freq = 300
    num_workers = 24
    #model = 'model/stage-1-paraphrase-multilingual-mpnet-base-v2-epochs-40-seq-50-tuned'
    #tokenizer = AutoTokenizer.from_pretrained(model)
    gradient_checkpointing = False
    num_cycles = 0.5
    warmup_ratio = 0.1
    epochs = 1
    encoder_lr = 1e-5
    decoder_lr = 1e-4
    eps = 1e-6
    betas = (0.9, 0.999)
    batch_size = 32
    weight_decay = 0.01
    max_grad_norm = 0.012
    #max_len = 512
    max_len = 128
    n_folds = 5
    seed = 42

In [31]:
# Define evaluation function
def evaluate(model, dataloader):
    model.eval()
    total_correct = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            total_correct += torch.sum(predictions == labels)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = total_correct / len(dataloader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return accuracy, f1

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_path, ignore_mismatched_sizes=True, num_labels=5)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path,ignore_mismatched_sizes=True)

# Load and preprocess data
train_texts = X_train.to_list() # list of training texts
train_labels =  y_train.to_list() # list of training labels (ints from 0 to num_labels-1)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels))

# Define evaluation function
def evaluate(model, dataloader):
    model.eval()
    total_correct = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            total_correct += torch.sum(predictions == labels)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = total_correct / len(dataloader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return accuracy, f1

# Fine-tune BERT model
model.train()
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,num_workers=0)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

num_train_steps = int(len(train_texts) / CFG.batch_size * CFG.epochs)
num_warmup_steps = num_train_steps * CFG.warmup_ratio
# Scheduler
#scheduler = get_cosine_schedule_with_warmup(
#    optimizer, 
#    num_warmup_steps = num_warmup_steps, 
#    num_training_steps = num_train_steps, 
#    num_cycles = CFG.num_cycles
#    )
total_epochs = []
epoch_val_score = []
for epoch in range(CFG.epochs):
    total_epochs.append(epoch)
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        #scheduler.step()
        running_loss += loss.item()
        if i % 50 == 49:    # Print every 50 batches
            print(f'Epoch {epoch+1}, Batch {i+1}/{len(train_loader)}, Loss: {running_loss/50:.4f}')
            running_loss = 0.0
    # Evaluate on validation set
    val_texts = X_val.to_list() # list of validation texts
    val_labels = y_val.to_list() # list of validation labels (ints from 0 to num_labels-1)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
    val_dataset = TensorDataset(
        torch.tensor(val_encodings['input_ids']),
        torch.tensor(val_encodings['attention_mask']),
        torch.tensor(val_labels))
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False,num_workers=0)
    val_accuracy, val_f1 = evaluate(model, val_loader)
    epoch_val_score.append(val_f1)
    print(f'Epoch {epoch+1}, Validation Accuracy: {val_accuracy:.5f}, Validation F1 macro score: {val_f1:.5f}')


In [ ]:
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [6]:
model.eval()
total_correct = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        total_correct += torch.sum(predictions == labels)
        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
accuracy = total_correct / len(val_loader.dataset)
f1 = f1_score(all_labels, all_preds, average='macro')

In [7]:
len(all_preds)

2475

In [8]:
target_dict.keys()

dict_keys(['OTHER', 'PROFANITY', 'SEXIST', 'RACIST', 'INSULT'])

In [9]:
print(classification_report(all_labels, all_preds,target_names=target_dict.keys(),digits=4))

              precision    recall  f1-score   support

       OTHER     0.9644    0.9260    0.9448       703
   PROFANITY     0.9303    0.9832    0.9560       475
      SEXIST     0.9458    0.9709    0.9582       413
      RACIST     0.9684    0.9803    0.9743       406
      INSULT     0.9136    0.8849    0.8990       478

    accuracy                         0.9455      2475
   macro avg     0.9445    0.9491    0.9465      2475
weighted avg     0.9456    0.9455    0.9452      2475

